# Goals: 
- Find out precision, recall, F1 for each tag class in predictions compared to test set on a token level and entity level.
- Can also use this to compare the unmodified test set to the modified test set

Must change out comparison set at top and bottom

In [1]:
import csv
#Evaluation of weakly supervised labeling
path_to_true_dataset = r"epi_gold_test.tsv"
#weakly supervised labels
path_to_pred_dataset = r'epi_test_setV4.tsv'

## Token-Level Evaluation
- This ignores differences between B-{tag} and I-{tag}

Import ground truth labels

In [2]:
#Load the corrected (trueified) test set data
# DO NOT CHANGE
true_tokens, true_tags= [],[]
with open(path_to_true_dataset,'r') as f:
    reader = csv.reader(f, delimiter="\t", quoting=csv.QUOTE_NONE)
    true_tokens = [row[0] for row in reader if len(row)>1]
    f.seek(0)
    true_tags = [row[1] for row in reader if len(row)>1]
f.close()

In [3]:
for x in range(10):
    print(true_tokens[x],true_tags[x])

Context O
Patients O
with O
pseudohypoparathyroidism B-DIS
type I-DIS
1b I-DIS
( O
PHP1b B-ABRV
) O
show O


Import the comparison set

In [4]:
#Load the weakly supervised labels (change delimiter)
import csv
with open(path_to_pred_dataset,'r') as f:
    reader = csv.reader(f, delimiter="\t")
    
    #for row in reader:
    #    if len(row)>1:
    #        print(row[0],row[1])
    #    else:
    #        print('')
    pred_tokens = [row[0] for row in reader if len(row)>1]
    f.seek(0)
    pred_tags = [row[1] for row in reader if len(row)>1]
f.close()

In [5]:
for x in range(10):
    print(pred_tokens[x],pred_tags[x])

Context O
Patients O
with O
pseudohypoparathyroidism B-DIS
type I-DIS
1b I-DIS
( O
PHP1b B-DIS
) O
show O


In [6]:
print(len(pred_tokens),len(true_tokens))
print(len(pred_tags),len(true_tags))
#Some of the sentences were combined when fixing the test set so there are now different number of sentences and different sizes of sentences

13910 13909
13910 13909


In [7]:
def compare_tag(pred_tag,true_tag):
    comparison_types = ["mismatch","skipped_annotations",
                    "loc_fn","loc_fp",
                    "stat_fn","stat_fp",
                    "date_fn","date_fp",
                    "ethn_fn","ethn_fp",
                    "sex_fn","sex_fp",
                    "dz_fn","dz_fp",
                    "abrv_fn","abrv_fp",
                    "epi_fn","epi_fp"]

    comparison = dict.fromkeys(comparison_types, 0)
    
    if pred_tag != true_tag:
        #This will fn everything, including ones that mismatch from I-LOC -> B-LOC, the others won't
        comparison["mismatch"]+=1
        
        #False positives
        if 'LOC' in pred_tag and 'LOC' not in true_tag:
            comparison["loc_fp"]+=1
        elif 'DIS' in pred_tag and 'DIS' not in true_tag:
            comparison["dz_fp"]+=1
        elif 'ABRV' in pred_tag and 'ABRV' not in true_tag:
            comparison["abrv_fp"]+=1
        elif 'DATE' in pred_tag and 'DATE' not in true_tag:
            comparison["date_fp"]+=1
        elif 'ETHN' in pred_tag and 'ETHN' not in true_tag:
            comparison["ethn_fp"]+=1
        elif 'SEX' in pred_tag and 'SEX' not in true_tag:
            comparison["sex_fp"]+=1
        elif 'STAT' in pred_tag and 'STAT' not in true_tag:
            comparison["stat_fp"]+=1
        elif 'EPI' in pred_tag and 'EPI' not in true_tag:
            comparison["epi_fp"]+=1
        
        #False negatives
        if 'LOC' in true_tag and 'LOC' not in pred_tag:
            comparison["loc_fn"]+=1
        elif 'DIS' in true_tag and 'DIS' not in pred_tag:
            comparison["dz_fn"]+=1
        elif 'ABRV' in true_tag and 'ABRV' not in pred_tag:
            comparison["abrv_fn"]+=1
        elif 'DATE' in true_tag and 'DATE' not in pred_tag:
            comparison["date_fn"]+=1
        elif 'ETHN' in true_tag and 'ETHN' not in pred_tag:
            comparison["ethn_fn"]+=1
        elif 'SEX' in true_tag and 'SEX' not in pred_tag:
            comparison["sex_fn"]+=1
        elif 'STAT' in true_tag and 'STAT' not in pred_tag:
            comparison["stat_fn"]+=1
        elif 'EPI' in true_tag and 'EPI' not in pred_tag:
            comparison["epi_fn"]+=1
        return comparison
    else:
        return comparison

Iterate through all tokens and labels but also accounts for small mismatches (off by one) in the tokens

In [8]:
efficacy_types = ["mismatch","skipped_annotations",
                    "loc_fn","loc_fp",
                    "stat_fn","stat_fp",
                    "date_fn","date_fp",
                    "ethn_fn","ethn_fp",
                    "sex_fn","sex_fp",
                    "dz_fn","dz_fp",
                    "abrv_fn","abrv_fp",
                    "epi_fn","epi_fp"]
efficacy_stats = dict.fromkeys(efficacy_types, 0)

true_idx, pred_idx=0,0

while pred_idx<len(pred_tokens) and true_idx<len(true_tokens):
    if true_tokens[true_idx] != pred_tokens[pred_idx]:
        if true_tokens[true_idx+1] == pred_tokens[pred_idx]:
            true_idx+=1
            #compare
            comparison = compare_tag(pred_tags[pred_idx],true_tags[true_idx])
            for entry in efficacy_types:
                efficacy_stats[entry]+=comparison[entry]
            pred_idx+=1
            true_idx+=1
        elif true_tokens[true_idx] == pred_tokens[pred_idx+1]:
            pred_idx+=1
            #compare
            comparison = compare_tag(pred_tags[pred_idx],true_tags[true_idx])
            for entry in efficacy_types:
                efficacy_stats[entry]+=comparison[entry]
            
            pred_idx+=1
            true_idx+=1
        
        else:
            print('uh oh-->','true_idx:',true_idx,', pred_idx:',pred_idx)
            print(true_tokens[true_idx],pred_tokens[pred_idx])
            efficacy_stats["skipped_annotations"]+=1
            pred_idx+=1
            true_idx+=1
            #skip compare
    else:
        #compare
        comparison = compare_tag(pred_tags[pred_idx],true_tags[true_idx])
        for entry in efficacy_types:
            efficacy_stats[entry]+=comparison[entry]
        
        pred_idx+=1
        true_idx+=1

uh oh--> true_idx: 2134 , pred_idx: 2134
RA RA.This
uh oh--> true_idx: 2135 , pred_idx: 2135
. study
uh oh--> true_idx: 2136 , pred_idx: 2136
This ,
uh oh--> true_idx: 2137 , pred_idx: 2137
study among
uh oh--> true_idx: 2138 , pred_idx: 2138
, a
uh oh--> true_idx: 2139 , pred_idx: 2139
among well
uh oh--> true_idx: 2140 , pred_idx: 2140
a -
uh oh--> true_idx: 2141 , pred_idx: 2141
well defined
uh oh--> true_idx: 2142 , pred_idx: 2142
- group
uh oh--> true_idx: 2143 , pred_idx: 2143
defined of
uh oh--> true_idx: 2144 , pred_idx: 2144
group atomic
uh oh--> true_idx: 2145 , pred_idx: 2145
of bomb
uh oh--> true_idx: 2146 , pred_idx: 2146
atomic survivors
uh oh--> true_idx: 2147 , pred_idx: 2147
bomb ,
uh oh--> true_idx: 2148 , pred_idx: 2148
survivors suggests
uh oh--> true_idx: 2149 , pred_idx: 2149
, that
uh oh--> true_idx: 2150 , pred_idx: 2150
suggests HTLV-1
uh oh--> true_idx: 2151 , pred_idx: 2151
that is
uh oh--> true_idx: 2152 , pred_idx: 2152
HTLV-1 associated
uh oh--> true_idx: 

uh oh--> true_idx: 2298 , pred_idx: 2301
and of
uh oh--> true_idx: 2299 , pred_idx: 2302
death hypovitaminosis
uh oh--> true_idx: 2300 , pred_idx: 2303
. C
uh oh--> true_idx: 2301 , pred_idx: 2304
The is
uh oh--> true_idx: 2302 , pred_idx: 2305
prevalence unknown
uh oh--> true_idx: 2303 , pred_idx: 2306
of in
uh oh--> true_idx: 2304 , pred_idx: 2307
hypovitaminosis Australian
uh oh--> true_idx: 2305 , pred_idx: 2308
C hospitalised
uh oh--> true_idx: 2306 , pred_idx: 2309
is patients
uh oh--> true_idx: 2307 , pred_idx: 2310
unknown ,
uh oh--> true_idx: 2308 , pred_idx: 2311
in and
uh oh--> true_idx: 2309 , pred_idx: 2312
Australian its
uh oh--> true_idx: 2310 , pred_idx: 2313
hospitalised clinical
uh oh--> true_idx: 2311 , pred_idx: 2314
patients relevance
uh oh--> true_idx: 2312 , pred_idx: 2315
, is
uh oh--> true_idx: 2313 , pred_idx: 2316
and uncertain
uh oh--> true_idx: 2314 , pred_idx: 2317
its .
uh oh--> true_idx: 2315 , pred_idx: 2318
clinical Aims
uh oh--> true_idx: 2316 , pred_

uh oh--> true_idx: 2488 , pred_idx: 2493
2 (
uh oh--> true_idx: 2490 , pred_idx: 2496
longer (
uh oh--> true_idx: 2491 , pred_idx: 2497
in interquartile
uh oh--> true_idx: 2492 , pred_idx: 2498
patients range
uh oh--> true_idx: 2493 , pred_idx: 2499
with (
uh oh--> true_idx: 2494 , pred_idx: 2500
hypovitaminosis IQR
uh oh--> true_idx: 2495 , pred_idx: 2501
C )
uh oh--> true_idx: 2496 , pred_idx: 2502
( 4
uh oh--> true_idx: 2497 , pred_idx: 2503
6 ,
uh oh--> true_idx: 2498 , pred_idx: 2504
days 8)
uh oh--> true_idx: 2499 , pred_idx: 2505
( vs
uh oh--> true_idx: 2500 , pred_idx: 2506
interquartile 4
uh oh--> true_idx: 2501 , pred_idx: 2507
range days
uh oh--> true_idx: 2504 , pred_idx: 2510
) 3
uh oh--> true_idx: 2507 , pred_idx: 2512
8) 6
uh oh--> true_idx: 2508 , pred_idx: 2513
vs )
uh oh--> true_idx: 2509 , pred_idx: 2514
4 ,
uh oh--> true_idx: 2510 , pred_idx: 2515
days P
uh oh--> true_idx: 2511 , pred_idx: 2516
( =
uh oh--> true_idx: 2512 , pred_idx: 2517
IQR 0.02
uh oh--> true_idx:

significant 47
uh oh--> true_idx: 2668 , pred_idx: 2673
reduction %
uh oh--> true_idx: 2669 , pred_idx: 2674
in to
uh oh--> true_idx: 2670 , pred_idx: 2675
the 20
uh oh--> true_idx: 2671 , pred_idx: 2676
prevalence %
uh oh--> true_idx: 2672 , pred_idx: 2677
of )
uh oh--> true_idx: 2673 , pred_idx: 2678
headaches but
uh oh--> true_idx: 2674 , pred_idx: 2679
( an
uh oh--> true_idx: 2675 , pred_idx: 2680
47 increase
uh oh--> true_idx: 2676 , pred_idx: 2681
% in
uh oh--> true_idx: 2677 , pred_idx: 2682
to the
uh oh--> true_idx: 2678 , pred_idx: 2683
20 prevalence
uh oh--> true_idx: 2679 , pred_idx: 2684
% of
uh oh--> true_idx: 2680 , pred_idx: 2685
) incoordination
uh oh--> true_idx: 2681 , pred_idx: 2686
but (
uh oh--> true_idx: 2682 , pred_idx: 2687
an 27
uh oh--> true_idx: 2683 , pred_idx: 2688
increase %
uh oh--> true_idx: 2684 , pred_idx: 2689
in to
uh oh--> true_idx: 2685 , pred_idx: 2690
the 47
uh oh--> true_idx: 2686 , pred_idx: 2691
prevalence %
uh oh--> true_idx: 2687 , pred_idx:

uh oh--> true_idx: 2870 , pred_idx: 2873
and with
uh oh--> true_idx: 2871 , pred_idx: 2874
the oral
uh oh--> true_idx: 2872 , pred_idx: 2875
various clefts
uh oh--> true_idx: 2873 , pred_idx: 2876
congenital (
uh oh--> true_idx: 2874 , pred_idx: 2877
malformations OC
uh oh--> true_idx: 2875 , pred_idx: 2878
associated )
uh oh--> true_idx: 2876 , pred_idx: 2879
with in
uh oh--> true_idx: 2877 , pred_idx: 2880
oral Italy
uh oh--> true_idx: 2878 , pred_idx: 2881
clefts across
uh oh--> true_idx: 2879 , pred_idx: 2882
( the
uh oh--> true_idx: 2880 , pred_idx: 2883
OC period
uh oh--> true_idx: 2881 , pred_idx: 2884
) 2001
uh oh--> true_idx: 2882 , pred_idx: 2885
in -
uh oh--> true_idx: 2883 , pred_idx: 2886
Italy 2014
uh oh--> true_idx: 2884 , pred_idx: 2887
across .
uh oh--> true_idx: 2885 , pred_idx: 2888
the Methods
uh oh--> true_idx: 2886 , pred_idx: 2889
period A
uh oh--> true_idx: 2887 , pred_idx: 2890
2001 retrospective
uh oh--> true_idx: 2888 , pred_idx: 2891
- analysis
uh oh--> true

...

uh oh--> true_idx: 5303 , pred_idx: 5302
than to
uh oh--> true_idx: 5304 , pred_idx: 5303
ever define
uh oh--> true_idx: 5305 , pred_idx: 5304
to a
uh oh--> true_idx: 5306 , pred_idx: 5305
define path
uh oh--> true_idx: 5307 , pred_idx: 5306
a to
uh oh--> true_idx: 5308 , pred_idx: 5307
path the
uh oh--> true_idx: 5309 , pred_idx: 5308
to clinic
uh oh--> true_idx: 5311 , pred_idx: 5311
clinic market
uh oh--> true_idx: 5312 , pred_idx: 5312
and .
uh oh--> true_idx: 5313 , pred_idx: 5313
the Twenty
uh oh--> true_idx: 5314 , pred_idx: 5314
market gene
uh oh--> true_idx: 5315 , pred_idx: 5315
. therapy
uh oh--> true_idx: 5316 , pred_idx: 5316
Twenty products
uh oh--> true_idx: 5317 , pred_idx: 5317
gene have
uh oh--> true_idx: 5318 , pred_idx: 5318
therapy already
uh oh--> true_idx: 5319 , pred_idx: 5319
products been
uh oh--> true_idx: 5320 , pred_idx: 5320
have approved
uh oh--> true_idx: 5321 , pred_idx: 5321
already and
uh oh--> true_idx: 5322 , pred_idx: 5322
been over
uh oh--> true_i

uh oh--> true_idx: 5514 , pred_idx: 5513
deposition brain
uh oh--> true_idx: 5515 , pred_idx: 5514
in ,
uh oh--> true_idx: 5516 , pred_idx: 5515
brain primarily
uh oh--> true_idx: 5517 , pred_idx: 5516
, affecting
uh oh--> true_idx: 5518 , pred_idx: 5517
primarily the
uh oh--> true_idx: 5519 , pred_idx: 5518
affecting basal
uh oh--> true_idx: 5520 , pred_idx: 5519
the ganglia
uh oh--> true_idx: 5521 , pred_idx: 5520
basal (
uh oh--> true_idx: 5522 , pred_idx: 5521
ganglia globus
uh oh--> true_idx: 5523 , pred_idx: 5522
( pallidus
uh oh--> true_idx: 5524 , pred_idx: 5523
globus mainly
uh oh--> true_idx: 5525 , pred_idx: 5524
pallidus )
uh oh--> true_idx: 5526 , pred_idx: 5525
mainly .
uh oh--> true_idx: 5527 , pred_idx: 5526
) The
uh oh--> true_idx: 5528 , pred_idx: 5527
. hallmark
uh oh--> true_idx: 5529 , pred_idx: 5528
The of
uh oh--> true_idx: 5530 , pred_idx: 5529
hallmark this
uh oh--> true_idx: 5531 , pred_idx: 5530
of syndrome
uh oh--> true_idx: 5532 , pred_idx: 5531
this is
uh 

biologic substrates
uh oh--> true_idx: 5725 , pred_idx: 5726
two ADSL
uh oh--> true_idx: 5726 , pred_idx: 5727
dephosphorylated enzyme
uh oh--> true_idx: 5727 , pred_idx: 5728
substrates :
uh oh--> true_idx: 5728 , pred_idx: 5729
of succinylaminoimidazole
uh oh--> true_idx: 5729 , pred_idx: 5730
ADSL carboxamide
uh oh--> true_idx: 5730 , pred_idx: 5731
enzyme riboside
uh oh--> true_idx: 5731 , pred_idx: 5732
: (
uh oh--> true_idx: 5732 , pred_idx: 5733
succinylaminoimidazole SAICAr
uh oh--> true_idx: 5733 , pred_idx: 5734
carboxamide )
uh oh--> true_idx: 5734 , pred_idx: 5735
riboside and
uh oh--> true_idx: 5736 , pred_idx: 5738
SAICAr S
uh oh--> true_idx: 5737 , pred_idx: 5739
) -
uh oh--> true_idx: 5738 , pred_idx: 5740
and Ado
uh oh--> true_idx: 5739 , pred_idx: 5741
succinyladenosine )
uh oh--> true_idx: 5740 , pred_idx: 5742
( .
uh oh--> true_idx: 5741 , pred_idx: 5743
S More
uh oh--> true_idx: 5742 , pred_idx: 5744
- than
uh oh--> true_idx: 5743 , pred_idx: 5745
Ado 80
uh oh--> t

uh oh--> true_idx: 5933 , pred_idx: 5932
their on
uh oh--> true_idx: 5934 , pred_idx: 5933
effects protein
uh oh--> true_idx: 5935 , pred_idx: 5934
on biogenesis
uh oh--> true_idx: 5936 , pred_idx: 5935
protein ,
uh oh--> true_idx: 5937 , pred_idx: 5936
biogenesis structural
uh oh--> true_idx: 5938 , pred_idx: 5937
, stability
uh oh--> true_idx: 5939 , pred_idx: 5938
structural and
uh oh--> true_idx: 5940 , pred_idx: 5939
stability activity
uh oh--> true_idx: 5941 , pred_idx: 5940
and as
uh oh--> true_idx: 5942 , pred_idx: 5941
activity well
uh oh--> true_idx: 5944 , pred_idx: 5943
well on
uh oh--> true_idx: 5945 , pred_idx: 5944
as purinosome
uh oh--> true_idx: 5946 , pred_idx: 5945
on assembly
uh oh--> true_idx: 5947 , pred_idx: 5946
purinosome were
uh oh--> true_idx: 5948 , pred_idx: 5947
assembly characterized
uh oh--> true_idx: 5949 , pred_idx: 5948
were .
uh oh--> true_idx: 5950 , pred_idx: 5949
characterized To
uh oh--> true_idx: 5951 , pred_idx: 5950
. date
uh oh--> true_idx: 5

uh oh--> true_idx: 6144 , pred_idx: 6143
granuloma was
uh oh--> true_idx: 6145 , pred_idx: 6144
annulare examined
uh oh--> true_idx: 6146 , pred_idx: 6145
was .
uh oh--> true_idx: 6147 , pred_idx: 6146
examined Confidence
uh oh--> true_idx: 6148 , pred_idx: 6147
. intervals
uh oh--> true_idx: 6149 , pred_idx: 6148
Confidence for
uh oh--> true_idx: 6150 , pred_idx: 6149
intervals prevalence
uh oh--> true_idx: 6151 , pred_idx: 6150
for and
uh oh--> true_idx: 6152 , pred_idx: 6151
prevalence incidence
uh oh--> true_idx: 6153 , pred_idx: 6152
and estimates
uh oh--> true_idx: 6154 , pred_idx: 6153
incidence were
uh oh--> true_idx: 6155 , pred_idx: 6154
estimates computed
uh oh--> true_idx: 6156 , pred_idx: 6155
were assuming
uh oh--> true_idx: 6157 , pred_idx: 6156
computed a
uh oh--> true_idx: 6158 , pred_idx: 6157
assuming binomial
uh oh--> true_idx: 6159 , pred_idx: 6158
a distribution
uh oh--> true_idx: 6160 , pred_idx: 6159
binomial using
uh oh--> true_idx: 6161 , pred_idx: 6160
distri

In [ ]:
for k,v in efficacy_stats.items():
    print(k,v)

In [9]:
#Get the number of true positives
def count_tags(tags):
    tag_types = ['LOC', 'ETHN', 'DATE', 'STAT', 'SEX','DIS','ABRV','EPI']
    tp = dict.fromkeys(tag_types, 0)
    
    for tag in tags:
        for tag_type in tag_types:
            if tag_type in tag:
                tp[tag_type]+=1
                ''' This is equivalent to doing:
                    ```
                    if 'LOC' in tag:
                        l+=1
                    elif 'DIS' in tag:
                        d+=1
                    elif 'ABRV' in tag:
                        a+=1
                    elif 'STAT' in tag:
                        s+=1
                    elif 'EPI' in tag:
                        e+=1
                    ```
                    in the previous code
                '''
    return tp

In [10]:
def get_stats(tp,fp,fn):
    if tp==0:
        return {'precision':0.0,'recall':0.0,'F1':0.0}
    else:
        precision = tp/(tp+fp)
        recall = tp/(tp+fn)
        F1=(2*precision*recall)/(precision+recall)
        return {'precision':precision,'recall':recall,'F1':F1}

In [11]:
#True Positives
tp = count_tags(true_tags)

#ALl stats
loc_stats = get_stats(tp["LOC"],efficacy_stats["loc_fp"],efficacy_stats["loc_fn"])
stat_stats = get_stats(tp["STAT"],efficacy_stats["stat_fp"],efficacy_stats["stat_fn"])
epi_stats = get_stats(tp["EPI"],efficacy_stats["epi_fp"],efficacy_stats["epi_fn"])
sex_stats = get_stats(tp["SEX"],efficacy_stats["sex_fp"],efficacy_stats["sex_fn"])
ethn_stats = get_stats(tp["ETHN"],efficacy_stats["ethn_fp"],efficacy_stats["ethn_fn"])
date_stats = get_stats(tp["DATE"],efficacy_stats["date_fp"],efficacy_stats["date_fn"])
dz_stats = get_stats(tp["DIS"],efficacy_stats["dz_fp"],efficacy_stats["dz_fn"])
abrv_stats = get_stats(tp["ABRV"],efficacy_stats["abrv_fp"],efficacy_stats["abrv_fn"])

#Overall
overall_tp = sum([val for val in tp.values()])

fp_types = [etype for etype in efficacy_types if 'fp' in etype]
overall_fp = sum([efficacy_stats[fp_type] for fp_type in fp_types])

fn_types = [etype for etype in efficacy_types if 'fn' in etype]
overall_fn = sum([efficacy_stats[fn_type] for fn_type in fn_types])

overall_stats = get_stats(overall_tp, overall_fp, overall_fn)

for k,v in overall_stats.items():
    print('token-level overall',k,v)
print('')
for k,v in dz_stats.items():
    print('disease classification',k,v)
print('')
for k,v in abrv_stats.items():
    print('abrv classification',k,v)
print('')
for k,v in loc_stats.items():
    print('location classification',k,v)
print('')
for k,v in date_stats.items():
    print('date classification',k,v)
print('')
for k,v in ethn_stats.items():
    print('ethnicity/nationality/race',k,v)
print('')
for k,v in sex_stats.items():
    print('biological sex classification',k,v)
print('')
for k,v in epi_stats.items():
    print('epidemiology type classification',k,v)
print('')
for k,v in stat_stats.items():
    print('statistic identification',k,v)
print('')
print('overall accuracy',1-(efficacy_stats["mismatch"])/(len(true_tags)-efficacy_stats["skipped_annotations"]))

token-level overall precision 0.9156626506024096
token-level overall recall 0.7067833698030634
token-level overall F1 0.7977770916949676

disease classification precision 0.927038626609442
disease classification recall 0.638109305760709
disease classification F1 0.7559055118110237

abrv classification precision 1.0
abrv classification recall 0.5836909871244635
abrv classification F1 0.7371273712737128

location classification precision 0.959349593495935
location classification recall 0.8027210884353742
location classification F1 0.8740740740740741

date classification precision 1.0
date classification recall 0.9411764705882353
date classification F1 0.9696969696969697

ethnicity/nationality/race precision 0.6956521739130435
ethnicity/nationality/race recall 1.0
ethnicity/nationality/race F1 0.8205128205128205

biological sex classification precision 1.0
biological sex classification recall 1.0
biological sex classification F1 1.0

epidemiology type classification precision 0.9747899159

In [12]:
sanity_check=0.0
for k,v in tp.items():
    print(k,v,'Relative weight:',v/overall_tp)
    sanity_check+=v/overall_tp
print('Sanity Check: ',sanity_check)

LOC 118 Relative weight: 0.0913312693498452
ETHN 16 Relative weight: 0.01238390092879257
DATE 96 Relative weight: 0.07430340557275542
STAT 206 Relative weight: 0.15944272445820434
SEX 36 Relative weight: 0.02786377708978328
DIS 432 Relative weight: 0.33436532507739936
ABRV 272 Relative weight: 0.21052631578947367
EPI 116 Relative weight: 0.08978328173374613
Sanity Check:  1.0


In [13]:
overall_tp

1292

## Entity-Level Evaluation (with seqeval)

Must re-input the comparison files because this is list of lists of tags, not just list of tags

In [14]:
#Import ground truth, do not change
y_true= []
#'epi_test_setV2-corrected.tsv'
with open(path_to_true_dataset,'r', encoding="utf-8") as f:
    reader = csv.reader(f, delimiter="\t")
    sentences_tags=[]
    for row in reader:
        if len(row)%2==0:
            if len(row)==0:
                y_true.append(sentences_tags.copy())
                sentences_tags.clear()
            else:
                sentences_tags.append(row[1])
f.close()

Import comparison set

In [15]:
#Load the weakly supervised labels (change delimiter)
y_pred= []
with open(path_to_pred_dataset,'r', encoding="utf-8") as f:
    reader = csv.reader(f, delimiter="\t")
    sentences_tags=[]
    for row in reader:
        if len(row)%2==0:
            if len(row)==0:
                y_pred.append(sentences_tags.copy())
                sentences_tags.clear()
            else:
                sentences_tags.append(row[1])
f.close()

In [16]:
print(len(y_true),len(y_pred))

537 534


In [17]:
#Correct the mismatch
for i in range(min(len(y_pred),len(y_true))):
    if len(y_pred[i]) != len(y_true[i]):
        print(len(y_pred[i]), len(y_pred[i+1]), len(y_true[i]))
        if len(y_pred[i+1])+len(y_pred[i]) == len(y_true[i]):
            y_pred[i]+=y_pred[i+1]
            y_pred.pop(i+1)
            print(len(y_pred[i]), len(y_true[i]))

36 30 16
30 11 22
11 18 30
18 20 11
20 34 18
34 17 20
17 13 34
13 19 17
19 34 13
34 17 19
17 27 16
27 25 18
25 61 17
61 60 28
60 10 25
10 15 43
15 19 18
19 26 60
26 11 10
11 11 15
11 28 19
28 32 26
32 30 11
30 20 11
20 23 28
23 32 32
32 39 30
39 11 20
11 39 23
39 68 32
68 27 39
27 46 11
46 40 39
40 21 68
21 42 27
42 20 46
20 20 40
40 40
18 18 21
18 26 42
26 50 20
50 66 20
66 51 18
51 56 18
56 17 26
17 25 50
25 29 66
29 34 51
34 17 56
16 31 25
31 22 29
22 19 34
19 44 17
44 18 16
18 13 31
31 31
37 31 22
31 43 19
43 16 44
16 25 18
25 11 13
11 30 37
30 31 31
31 52 43
52 62 16
62 43 25
43 44 11
44 52 30
52 31 31
31 9 52
9 31 62
31 14 43
14 29 44
29 25 52
25 39 31
39 22 9
22 31 31
31 55 14
55 22 29
22 32 25
32 22 39
36 23 31
23 9 55
9 53 22
53 27 32
27 44 22
44 18 36
18 14 23
14 15 9
15 25 53
25 11 27
11 20 44
20 26 18
26 19 14
19 20 15
20 9 25
9 11 11
11 19 20
19 16 26
16 34 19
34 28 20
28 12 9
12 21 11
21 14 19
14 17 16
17 32 34
32 17 28
17 31 12
31 23 21
23 20 14
20 32 17
15 21 17
21 22 3

IndexError: list index out of range

In [18]:
print(len(y_true),len(y_pred))

537 527


In [19]:
from seqeval.metrics import f1_score, precision_score, recall_score
def get_seqevalmetrics(y_true,y_pred):
    return {
            "precision": precision_score(y_true, y_pred),
            "recall": recall_score(y_true, y_pred),
            "f1": f1_score(y_true, y_pred),
        }

In [20]:
print('OVERALL',get_seqevalmetrics(y_true,y_pred))

ValueError: Found input variables with inconsistent numbers of samples:
[25, 20, 12, 61, 5, 8, 26, 18, 48, 18, 23, 22, 19, 34, 23, 11, 24, 21, 15, 15, 24, 15, 28, 2, 25, 22, 17, 21, 5, 28, 27, 12, 5, 22, 26, 28, 14, 26, 53, 5, 19, 9, 15, 13, 26, 22, 22, 10, 14, 27, 50, 33, 20, 18, 9, 40, 28, 44, 24, 17, 26, 22, 27, 22, 32, 35, 18, 14, 42, 21, 17, 20, 19, 21, 32, 29, 34, 20, 59, 13, 18, 22, 38, 53, 20, 39, 18, 13, 27, 36, 16, 22, 30, 11, 18, 20, 34, 17, 13, 19, 16, 18, 17, 28, 25, 43, 18, 60, 10, 15, 19, 26, 11, 11, 28, 32, 30, 20, 23, 32, 39, 11, 39, 68, 27, 46, 40, 21, 42, 20, 20, 18, 18, 26, 50, 66, 51, 56, 17, 25, 29, 34, 17, 16, 31, 22, 19, 44, 18, 13, 37, 31, 43, 16, 25, 11, 30, 31, 52, 62, 43, 44, 52, 31, 9, 31, 14, 29, 25, 39, 22, 31, 55, 22, 32, 22, 36, 23, 9, 53, 27, 44, 18, 14, 15, 25, 11, 20, 26, 19, 20, 9, 11, 19, 16, 34, 28, 12, 21, 14, 17, 32, 17, 31, 23, 20, 32, 15, 21, 22, 18, 27, 12, 27, 29, 41, 37, 15, 13, 38, 37, 38, 14, 29, 33, 19, 15, 38, 32, 42, 33, 27, 13, 46, 26

### Get individual entity types

This will normalize all other entities except for a single one

In [21]:
from copy import deepcopy
#LOCATION
y_true_loc = deepcopy(y_true)
y_pred_loc = deepcopy(y_pred)
for i in range(len(y_pred)):
    for j in range(len(y_pred[i])):
        if 'LOC' not in y_pred[i][j]:
            y_pred_loc[i][j] = 'O'
        if 'LOC' not in y_true[i][j]:
            y_true_loc[i][j] = 'O'

IndexError: list index out of range

In [22]:
#EPI IDENTIFIER
y_true_epi = deepcopy(y_true)
y_pred_epi = deepcopy(y_pred)
for i in range(len(y_pred)):
    for j in range(len(y_pred[i])):
        if 'EPI' not in y_true[i][j]:
            y_true_epi[i][j] = 'O'
        if 'EPI' not in y_pred[i][j]:
            y_pred_epi[i][j] = 'O'

IndexError: list index out of range

In [ ]:
#EPI Rate
y_true_stat = deepcopy(y_true)
y_pred_stat = deepcopy(y_pred)
for i in range(len(y_pred)):
    for j in range(len(y_pred[i])):
        if 'STAT' not in y_true[i][j]:
            y_true_stat[i][j] = 'O'
        if 'STAT' not in y_pred[i][j]:
            y_pred_stat[i][j] = 'O'

In [ ]:
#DATE
y_true_date = deepcopy(y_true)
y_pred_date = deepcopy(y_pred)
for i in range(len(y_pred)):
    for j in range(len(y_pred[i])):
        if 'DATE' not in y_true[i][j]:
            y_true_date[i][j] = 'O'
        if 'DATE' not in y_pred[i][j]:
            y_pred_date[i][j] = 'O'

In [ ]:
#SEX
y_true_sex = deepcopy(y_true)
y_pred_sex = deepcopy(y_pred)
for i in range(len(y_pred)):
    for j in range(len(y_pred[i])):
        if 'SEX' not in y_true[i][j]:
            y_true_sex[i][j] = 'O'
        if 'SEX' not in y_pred[i][j]:
            y_pred_sex[i][j] = 'O'

In [ ]:
#ETHN
y_true_ethn = deepcopy(y_true)
y_pred_ethn = deepcopy(y_pred)
for i in range(len(y_pred)):
    for j in range(len(y_pred[i])):
        if 'ETHN' not in y_true[i][j]:
            y_true_ethn[i][j] = 'O'
        if 'ETHN' not in y_pred[i][j]:
            y_pred_ethn[i][j] = 'O'

In [ ]:
#DIS
y_true_dz = deepcopy(y_true)
y_pred_dz = deepcopy(y_pred)
for i in range(len(y_pred)):
    for j in range(len(y_pred[i])):
        if 'DIS' not in y_true[i][j]:
            y_true_dz[i][j] = 'O'
        if 'DIS' not in y_pred[i][j]:
            y_pred_dz[i][j] = 'O'

In [ ]:
#ABRV
y_true_abrv = deepcopy(y_true)
y_pred_abrv = deepcopy(y_pred)
for i in range(len(y_pred)):
    for j in range(len(y_pred[i])):
        if 'ABRV' not in y_true[i][j]:
            y_true_abrv[i][j] = 'O'
        if 'ABRV' not in y_pred[i][j]:
            y_pred_abrv[i][j] = 'O'

In [ ]:
#Get all as a single printout
entity_level = {'OVERALL':get_seqevalmetrics(y_true,y_pred),
                'LOCATION':get_seqevalmetrics(y_true_loc,y_pred_loc),
                'EPI IDENTIFIER':get_seqevalmetrics(y_true_epi,y_pred_epi),
                'EPI RATE':get_seqevalmetrics(y_true_stat,y_pred_stat),
                'DATE':get_seqevalmetrics(y_true_date,y_pred_date),
                'SEX':get_seqevalmetrics(y_true_sex,y_pred_sex),
                'ETHN':get_seqevalmetrics(y_true_ethn,y_pred_ethn),
                'DIS':get_seqevalmetrics(y_true_dz,y_pred_dz),
                'ABRV':get_seqevalmetrics(y_true_abrv,y_pred_abrv)}

for typ,metrics in entity_level.items():
    for stat,num in metrics.items():
        print(typ,stat,num)
    print()